<a href="https://colab.research.google.com/github/Chaithanya3K/smart-food-waste-detection/blob/main/Vision_Transformer_for_calculating_food_waste_percentage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import timm
import torch
import torch.nn as nn

device = "cuda" if torch.cuda.is_available() else "cpu"

vit = timm.create_model(
    "vit_base_patch16_224.dino",
    pretrained=True
)
vit.head = nn.Identity()
vit = vit.to(device)
vit.eval()


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/343M [00:00<?, ?B/s]

VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (patch_drop): Identity()
  (norm_pre): Identity()
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (q_norm): Identity()
        (k_norm): Identity()
        (attn_drop): Dropout(p=0.0, inplace=False)
        (norm): Identity()
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): Identity()
      (drop_path1): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False

In [2]:
import cv2
import numpy as np

IMAGENET_MEAN = np.array([0.485, 0.456, 0.406])
IMAGENET_STD  = np.array([0.229, 0.224, 0.225])

def vit_preprocess(masked_img):
    img = cv2.resize(masked_img, (224, 224))
    img = img / 255.0
    img = (img - IMAGENET_MEAN) / IMAGENET_STD
    img = torch.tensor(img, dtype=torch.float32).permute(2,0,1).unsqueeze(0)
    return img.to(device)


In [6]:
before_food = cv2.imread("/content/drive/MyDrive/before.png")
after_food  = cv2.imread("/content/drive/MyDrive/before.png")


In [7]:
with torch.no_grad():
    f_before = vit(vit_preprocess(before_food))
    f_after  = vit(vit_preprocess(after_food))


In [8]:
import torch.nn.functional as F

similarity = F.cosine_similarity(f_before, f_after).item()
change_score = 1 - similarity     # 0 → no change, 1 → large change

# Convert to percentage
food_waste_percentage = change_score * 100
food_waste_percentage = max(0, min(food_waste_percentage, 100))

print(f"Estimated Food Waste (ViT-based): {food_waste_percentage:.2f}%")


Estimated Food Waste (ViT-based): 0.00%
